In [ ]:
!pip install num2words
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from num2words import num2words
import keras
from keras.layers import Input
from keras.models import Model
import nltk
from nltk.data import load
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
import re
from textblob import TextBlob
from keras.layers import Bidirectional, SpatialDropout1D,Concatenate
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('omw-1.4')
from collections import Counter
import tensorflow as tf
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ReduceLROnPlateau
import io
import string
from sklearn.metrics import mean_squared_error
import math
import sklearn
import gc
#import libraries

In [ ]:
!pip install transformers==4.5.1

In [ ]:
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import Model, Input, backend as K
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from transformers import TFBertModel, BertConfig, BertTokenizerFast

In [ ]:
#turn on gpu for faster computations
gpu = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu))
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
model_name = 'bert_v13'

data_dir = Path('../input/commonlitreadabilityprize')
train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'
pretrained_dir = '../input/tfbert-large-uncased'
# just a variables

In [ ]:
trn = pd.read_csv(train_file, index_col='id')
tst = pd.read_csv(test_file, index_col='id')
y = trn['target'].values
print(trn.shape, y.shape, tst.shape)
trn.head()
# view of file

In [ ]:
tf.__version__

In [ ]:
trn['excerpt'].str.split(' ').apply(len).describe()

In [ ]:
trn['excerpt'].str.split(' ').apply(len).hist(bins=50)

In [ ]:
from distutils.dir_util import copy_tree

copy_tree("../input/bertconfig", "/kaggle/working/")
# copy config file necessary for a BERTConfig

In [ ]:
# encode bert model from tokenizer
def bert_encode(texts, tokenizer, max_len=205):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    
    for text in texts:
        token = tokenizer(text, max_length=max_len, truncation=True, padding='max_length',
                         add_special_tokens=True)
        input_ids.append(token['input_ids'])
        token_type_ids.append(token['token_type_ids'])
        attention_mask.append(token['attention_mask'])
    
    return np.array(input_ids), np.array(token_type_ids), np.array(attention_mask)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-large-uncased")
tokenizer.save_pretrained("/kaggle/working")
# save pretrained tokenizer

In [ ]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk("/kaggle/working"):
    f.extend(filenames)
    break

In [ ]:
f

In [ ]:
bert_config = BertConfig.from_pretrained("/kaggle/working")
bert_config.output_hidden_states = True

X = bert_encode(trn['excerpt'].values, tokenizer, max_len=205)
X_tst = bert_encode(tst['excerpt'].values, tokenizer, max_len=205)
y = trn['target'].values
print(X[0].shape, X_tst[0].shape, y.shape)
# load BertConfig from config file
# encode bert from train file

In [ ]:
# model building function
def build_model(bert_model, max_len=205):    
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    token_type_ids = Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    sequence_output = bert_model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    clf_output = sequence_output[:, 0, :]
    clf_output = Dropout(.1)(clf_output)
    out = Dense(1, activation='linear')(clf_output)
    
    model = Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss="mean_squared_error", metrics=[RootMeanSquaredError()])
    
    return model

In [ ]:
# just a scheduler getting sure epochs goes one after another
def scheduler(epoch, lr, warmup=5, decay_start=10):
    if epoch <= warmup:
        return lr / (warmup - epoch + 1)
    elif warmup < epoch <= decay_start:
        return lr
    else:
        return lr * tf.math.exp(-.1)

In [ ]:
ls = tf.keras.callbacks.LearningRateScheduler(scheduler)
es = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

cv = sklearn.model_selection.KFold(n_splits=5, shuffle=True, random_state=42)

p = np.zeros_like(y, dtype=float)
p_tst = np.zeros((X_tst[0].shape[0], ), dtype=float)

In [ ]:
# training 5 models (it has to be divided, because there is too much data)
for i, (i_trn, i_val) in enumerate(cv.split(X[0]), 1):
    print(f'training CV #{i}:')
    tf.random.set_seed(42 + i)

    bert_model = TFBertModel.from_pretrained("bert-large-uncased", config=bert_config)
    clf = build_model(bert_model, max_len=205)
    if i == 1:
        print(clf.summary())
    history = clf.fit([x[i_trn] for x in X], y[i_trn],
                      validation_data=([x[i_val] for x in X], y[i_val]),
                      epochs=9,
                      batch_size=8,
                      callbacks=[ls])
    clf.save_weights(f'{model_name}_cv{i}.h5')

    p[i_val] = clf.predict([x[i_val] for x in X]).flatten()
    p_tst += clf.predict(X_tst).flatten() / 5
    
    tf.keras.backend.clear_session()
    del clf, bert_model
    gc.collect()

In [ ]:
sub = pd.read_csv(sample_file, index_col='id')
sub['target'] = p_tst
sub.head()

In [ ]:
# graph plotting function
def graph_plots(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
graph_plots(history, "root_mean_squared_error")